In [17]:
import ollama
response = ollama.chat(model='benedict/linkbricks-llama3.1-korean:70b', messages=[
  {
    'role': 'user',
    'content': '안녕',
  },
])
print(response['message']['content'])

안녕하세요, 저는 링크브릭스 호라이즌 AI입니다. 저는 사용자의 삶을 편리하게 만들고 지원해주는 인공지능 어시스턴트입니다. 어떤 도움이 필요하십니까?


In [32]:
import pandas as pd
import ollama
import os

# 경로 설정
input_dir = "./Data/None"
output_dir = "./Data/GR"
os.makedirs(output_dir, exist_ok=True)  # 출력 디렉토리 생성

# 데이터 생성 함수
def generate_data(hate_type, missing_count, existing_data):
    while missing_count > 0:
        # Train 데이터에서 해당 클래스의 랜덤 예시 3개 추출
        examples = existing_data[existing_data['hate'] == hate_type]['comments'].dropna().sample(3).tolist()
        prompt = f"""
        아래의 형식에 맞추어 {hate_type} 유형의 새로운 문장을 하나 작성해줘.
        **출력 형식: {{새로운 문장}}**
        다른 형식의 출력은 포함하지 말아줘.

        예시:
        1. {{ {examples[0]} }}
        2. {{ {examples[1]} }}
        3. {{ {examples[2]} }}

        이제 새로운 문장을 작성해줘.
        """

        # Ollama 호출
        response = ollama.chat(model="benedict/linkbricks-llama3.1-korean:70b", messages=[
            {
                "role": "user",
                "content": prompt.strip(),
            }
        ])
        generated_text = response['message']['content'].strip()

        # 출력 형식 검증
        if not (generated_text.startswith("{") and generated_text.endswith("}")):
            continue

        # 형식이 올바른 경우 저장
        new_row = {"comments": generated_text[1:-1], "hate": hate_type}
        existing_data = pd.concat([existing_data, pd.DataFrame([new_row])], ignore_index=True)
        missing_count -= 1

    return existing_data

# 각 파일 처리
for file_name in sorted(os.listdir(input_dir)):
    if file_name.endswith(".csv"):
        input_path = os.path.join(input_dir, file_name)
        output_file = f"GR_{file_name.split('_')[-1]}"
        output_path = os.path.join(output_dir, output_file)

        # 파일 로드
        gr_data = pd.read_csv(input_path)

        # 클래스별 최대 개수 설정
        target_count = gr_data['hate'].value_counts().max()

        # 현재 데이터 분포 확인
        current_counts = gr_data['hate'].value_counts()

        # 부족한 클래스 데이터 생성
        for hate_type in ["offensive", "hate"]:
            current_count = current_counts.get(hate_type, 0)
            missing_count = target_count - current_count  # 부족한 개수 계산
            while missing_count > 0:
                gr_data = generate_data(hate_type, missing_count, gr_data)
                current_counts = gr_data['hate'].value_counts()  # 생성 후 분포 업데이트
                missing_count = target_count - current_counts.get(hate_type, 0)

        # 파일 저장
        gr_data.to_csv(output_path, index=False, encoding="utf-8-sig")

        # 클래스 균형 확인
        final_counts = gr_data['hate'].value_counts()
        print(f"{output_path} - 모든 클래스의 데이터 균형이 맞습니다. 최종 분포: {final_counts}")

# 최종 완료 메시지
print("모든 파일에 대한 데이터 생성 및 저장이 완료되었습니다.")


./Data/GR/GR_1.csv - 모든 클래스의 데이터 균형이 맞습니다. 최종 분포: hate
none         44
offensive    44
hate         44
Name: count, dtype: int64
./Data/GR/GR_2.csv - 모든 클래스의 데이터 균형이 맞습니다. 최종 분포: hate
offensive    47
none         47
hate         47
Name: count, dtype: int64
./Data/GR/GR_3.csv - 모든 클래스의 데이터 균형이 맞습니다. 최종 분포: hate
hate         42
none         42
offensive    42
Name: count, dtype: int64
./Data/GR/GR_4.csv - 모든 클래스의 데이터 균형이 맞습니다. 최종 분포: hate
none         50
hate         50
offensive    50
Name: count, dtype: int64
./Data/GR/GR_5.csv - 모든 클래스의 데이터 균형이 맞습니다. 최종 분포: hate
offensive    40
none         40
hate         40
Name: count, dtype: int64
모든 파일에 대한 데이터 생성 및 저장이 완료되었습니다.


In [33]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import f1_score
import os

# 사용자 정의 데이터셋 클래스
class SimpleDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "label": torch.tensor(self.labels[idx], dtype=torch.long),
        }

# 경로 설정
train_dir = "./Data/GR"  # None을 SC로 변경
test_file = "./Data/Test.csv"
summary_file = "./Data/Summary.csv"

# 모델 및 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)  # 3개 클래스
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 학습 및 평가 결과 저장
results = []

# 테스트 데이터 준비
test_data = pd.read_csv(test_file)
test_texts = test_data["comments"].tolist()
test_labels = test_data["hate"].map({"none": 0, "hate": 1, "offensive": 2}).tolist()
test_dataset = SimpleDataset(test_texts, test_labels, tokenizer, max_len=128)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 학습 루프
for train_file in sorted(os.listdir(train_dir)):
    if train_file.endswith(".csv"):
        train_data = pd.read_csv(os.path.join(train_dir, train_file))
        train_texts = train_data["comments"].tolist()
        train_labels = train_data["hate"].map({"none": 0, "hate": 1, "offensive": 2}).tolist()

        train_dataset = SimpleDataset(train_texts, train_labels, tokenizer, max_len=128)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        # 옵티마이저 설정
        optimizer = AdamW(model.parameters(), lr=5e-5)

        # 학습
        model.train()
        for epoch in range(3):  # 3 에포크
            total_loss = 0
            for batch in train_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["label"].to(device)

                optimizer.zero_grad()
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            print(f"{train_file} - Epoch {epoch + 1} - Loss: {total_loss:.4f}")

        # 평가
        model.eval()
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for batch in test_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["label"].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, axis=1).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())

        f1_micro = f1_score(all_labels, all_preds, average="micro")
        f1_macro = f1_score(all_labels, all_preds, average="macro")
        print(f"Test Results for {train_file}: F1 Micro = {f1_micro:.4f}, F1 Macro = {f1_macro:.4f}")
        results.append({"File": train_file, "F1 Micro": f1_micro, "F1 Macro": f1_macro})

# 결과 저장 또는 추가
if os.path.exists(summary_file):
    # 기존 파일이 있는 경우: 기존 데이터를 읽어서 새로운 데이터를 추가
    existing_results = pd.read_csv(summary_file)
    results_df = pd.concat([existing_results, pd.DataFrame(results)], ignore_index=True)
else:
    # 기존 파일이 없는 경우: 새로운 데이터만 저장
    results_df = pd.DataFrame(results)

results_df.to_csv(summary_file, index=False)
print(f"Summary updated and saved to {summary_file}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


GR_1.csv - Epoch 1 - Loss: 5.6620
GR_1.csv - Epoch 2 - Loss: 5.4239
GR_1.csv - Epoch 3 - Loss: 5.4416
Test Results for GR_1.csv: F1 Micro = 0.3673, F1 Macro = 0.2975


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


GR_2.csv - Epoch 1 - Loss: 5.6970
GR_2.csv - Epoch 2 - Loss: 5.5288
GR_2.csv - Epoch 3 - Loss: 5.5013
Test Results for GR_2.csv: F1 Micro = 0.4034, F1 Macro = 0.1953


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


GR_3.csv - Epoch 1 - Loss: 4.4310
GR_3.csv - Epoch 2 - Loss: 4.3951
GR_3.csv - Epoch 3 - Loss: 4.1675
Test Results for GR_3.csv: F1 Micro = 0.3992, F1 Macro = 0.3856


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


GR_4.csv - Epoch 1 - Loss: 5.1806
GR_4.csv - Epoch 2 - Loss: 4.9747
GR_4.csv - Epoch 3 - Loss: 4.7198
Test Results for GR_4.csv: F1 Micro = 0.3715, F1 Macro = 0.3719


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


GR_5.csv - Epoch 1 - Loss: 4.5588
GR_5.csv - Epoch 2 - Loss: 4.2146
GR_5.csv - Epoch 3 - Loss: 4.1156
Test Results for GR_5.csv: F1 Micro = 0.3439, F1 Macro = 0.3303
Summary updated and saved to ./Data/Summary.csv
